# Worksheet 10 Solutions

## MCS 260 Fall 2020 - David Dumas

## Instructions:
* Complete the problems below in preparation for Quiz 10.
* Collaboration is strongly encouraged on Worksheets
* When working in discussion, please have someone in your work group share a screen.
* Test your work to make sure it does what is asked
* It is not expected that you will complete these problems in the Tue/Thu discussion meeting.

## Problem 1

As with sequences, assigning a key-value pair is handled by the special method `__setitem__`.  For example,

```python
obj["foo"] = "bar"
```

becomes the method call

```python
obj.__setitem__("foo","bar")
```

Create a subclass of the built-in type `dict` called `WordDict` that will only allow keys that are words, meaning they are strings (instances of `str`) that consist entirely of the characters `A` to `Z` and `a` to `z`.

Specifically, your class should raise a `TypeError` if a key is not an instance of `str`, and should raise a `ValueError` if the key contains characters other the ones allowed.

Note: Once you have determined a key is allowed, you should let the `__setitem__` method of the superclass `dict` do the work.  For this, the `super()` function will be useful.

If you've completed this problem correctly, it should behave as in the following REPL session.  (Note you may need to change `WordDict` to `modulename.WordDict` if you import the class from a module called `modulename`.)

```
>>> d = WordDict()
>>> d[1]=2
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "<stdin>", line 4, in __setitem__
TypeError: Only keys that are instances of str are allowed
>>> d["foo"]="bar"
>>> d["two words"]=81
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "<stdin>", line 6, in __setitem__
ValueError: Key 'two words' contains non-alphabet characters
>>> d["other"]=99
>>> d
{'foo': 'bar', 'other': 99}
>>> isinstance(d,dict)  # It subclasses dict, so it is an instance!
True
>>>
```

In [ ]:
class WordDict(dict):
    '''Dictionary which only accepts words as keys'''
    
    def __setitem__(self, key, value):
        '''Allows a new key to be added, but only if its a string with only alpha characters'''
        if not isinstance(key, str):
            raise TypeError("Key must be a string")
        if not key.isalpha():
            raise ValueError("Key must consist of alphabet characters ONLY")
        
        # Once you know the key is an alpha string, add it using the dict __setitem__
        super().__setitem__(key,value)

## Problem 2

In Lecture 26 we built a module `gs` that defines a class `FiniteGeometricSeries` that represents and lazily evaluates a finite geometric series.  The source code is here:

* [gs.py](https://github.com/daviddumas/mcs260fall2020/blob/master/samplecode/gs/gs.py)

However, the class we wrote in lecture doesn't allow indexing with negative numbers, e.g. where `[-1]` means the last element, etc..

Modify `FiniteGeometricSeries` so that it does support negative indices in the same way that `list` does.  However, negative indices that are too large in magnitude to represent an item in the series should still raise an exception.

Below is a transcript of a test of the updated module in the REPL.  Your class should behave in the same way after the modifications.

```
>>> import gs
>>> S = gs.FiniteGeometricSequence(start=2,ratio=3,length=5)
>>> S[4]
162
>>> S[-1]
162
>>> S[-2]
54
>>> S[-500]
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "/home/ddumas/Dropbox/teaching/mcs260/samplecode/gs/gs_practice.py", line 23, in __getitem__
    raise IndexError("index out of range for series with {} terms".format(
IndexError: index out of range for series with 5 terms
```

In [ ]:
"""Lazy finite geometric sequences"""
# Changed the __getitem

class FiniteGeometricSequence:
    """Finite geometric sequence that computes
    terms only as needed (is "lazy")
    """
    def __init__(self,start,ratio,length):
        """Initialize geometric sequence with first term
        `start`, ratio of successive terms `ratio` and
        total number of terms `length`.
        """
        self.start = start
        self.ratio = ratio
        self.length = length

    def __getitem__(self,idx):
        """Compute a term in the geometric sequence with idx in range(-self.length, self.length)"""

        # If idx is negative, attempt to convert it to the corresponding positive value
        # By changing idx that's negative, then this math works as expected
        if idx < 0:
            idx += self.length

        # If idx is still negative, then it's out of range. Same if idx >=self.length.
        if idx < 0 or idx >= self.length:
            raise IndexError("index not valid for geometric sequence of length {}".format(
                self.length  
            ))
        
        
        return self.start * self.ratio**idx
        # Reminder: PEMDAS means this evaluates as
        # self.start * (self.ratio**idx)

    def __str__(self):
        """Human-readable string representation"""
        return "{}(start={},ratio={},length={})".format(
            self.__class__.__name__,
            self.start,
            self.ratio,
            self.length
        )

    def __setitem__(self,idx,val):
        """Modify either the start or ratio using
        item assignment
        """
        if idx==0:
            # change start
            self.start = val
        else:
            # change the ratio so that
            # the term at index idx becomes
            # val
            self.ratio = (val/self.start)**(1/idx)

    def __repr__(self):
        """Unambiguous string representation"""
        return str(self)

    def __len__(self):
        """Number of terms in the sequence"""
        return self.length

## Problem 3

Recall that a palindrome is a word (or string) that is unchanged after it is reversed.

Here is a description of a palindrome that suggests a recursive way of checking whether a string is a palindrome or not:
* Any string with 0 or 1 characters is a palindrome
* A string with at least 2 characters is a palindrome if both of these conditions are met:
    * The first and last characters are equal
    * After the first and last characters are removed, what remains is a palindrome
    
Use this description to write a function `is_palindrome_recursive(s)` that returns `True` if string `s` is a palindrome, and `False` otherwise.

In [2]:
def is_palindrome_recursive(s):
    '''Determines whether a string is a palindrom recursively'''
    # Base case: all strings of length 0 or 1 are palindromes
    if len(s) == 0:
        return True
    if len(s) == 1:
        return True
    # otherwise, check that the first and last are equal, then
    # recurse on the 'middle' string that's left
    return s[0]==s[-1] and is_palindrome_recursive(s[1:-1])

True
True
False
True


## Problem 4

The Fibonacci numbers are the sequence of positive integers $F_i$ defined as follows:

* $F_0 = 0$ and $F_1 = 1$
* For $i > 1$ we have $F_i = F_{i-1} + F_{i-2}$.

Since this definition of $F_i$ involves other Fibonacci numbers, it naturally lends itself to recursion.  Write a function `fib(n)` that calculates $F_n$ using recursion.

Test your function to ensure that it produces the following list:

```
n      0  1  2  3  4  5  6  7   8   9   10
fib(n) 0  1  1  2  3  5  8  13  21  34  55
```

In [9]:
def fib(n):
    '''Calculates the nth fibonacci number recursively'''
    # Base case: fib(0)=0 and fib(1)=1
    if n == 0:
        return 0
    if n == 1:
        return 1
    
    # If n > 1, recurse on n-1 and n-2
    return fib(n-1) + fib(n-2)

## Problem 5

This problem builds on the previous one, and is relatively challenging.

When you call `fib(n)`, how many times does the `fib` function get called in total?

To start exploring this, you might first add a `print()` statement in the body of `fib(n)` so that you can see how many calls are made.  After trying this out a few times, can you see a pattern?  Can you analyze it theoretically to find a formula?

The answer should be a formula involving `n`, and it is acceptable to also have `fib(n)` in the formula.

Answer:
Define the sequence $G_n$ as the number of times that the function `fib(n)` is called for an input $n$.

By modifying our original function to print out a message every time the code executes, we can find the pattern:
```
n F_n G_n
0 0 1
1 1 1
2 1 3
3 2 5
4 3 9
5 5 15
6 8 25
7 13 41
8 21 67
9 34 109
10 55 177
```
So defined recursively, we have $G_0 = 1$, $G_1 = 1$, and $G_n = 1 + G_{n-1} + G_{n-2}$.